In [104]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.stem.lancaster import LancasterStemmer
import json
import pickle
import numpy as np
import pandas as pd
import random
import operator
import warnings
warnings.filterwarnings('ignore')
import spacy
nlp = spacy.load('en_core_web_sm')
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, PReLU, Flatten, Softmax, Activation, Dropout
from tensorflow.keras.models import Model
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split as tts
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder as LE
from sklearn.metrics.pairwise import cosine_similarity

In [105]:
# list of all words
words=[]
# list of all category of intents
classes = []
documents = []
json_file_name = './intents-generated.json'
csv_file_name = './mydata1.csv'
data = pd.read_csv(csv_file_name)
# data = data.sample(1000)
myintents = open(json_file_name,encoding="utf-8").read()
intents = json.loads(myintents)

In [106]:

for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to the classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [107]:
stop_words = set(stopwords.words('english'))
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in stop_words]
words = sorted(list(set(words)))
# print("words",words)
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
# print (len(documents), "documents")
# classes = intents
# print (len(classes), "classes", classes)
# words = all words, vocabulary
# print (len(words), "unique lemmatized words", words)

# pickle.dump(words,open('words.pkl','wb'))
# pickle.dump(classes,open('classes.pkl','wb'))

In [108]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [110]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=10, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/5
6085/6085 [==============================] - 17s 3ms/sample - loss: 1.3033 - accuracy: 0.4460
Epoch 2/5
6085/6085 [==============================] - 15s 2ms/sample - loss: 0.9829 - accuracy: 0.5915
Epoch 3/5
6085/6085 [==============================] - 15s 2ms/sample - loss: 0.7657 - accuracy: 0.7019
Epoch 4/5
6085/6085 [==============================] - 15s 2ms/sample - loss: 0.6327 - accuracy: 0.7574
Epoch 5/5
6085/6085 [==============================] - 13s 2ms/sample - loss: 0.5191 - accuracy: 0.8131
model created


In [ ]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
#     print(len(words))
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag:",w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [ ]:
predict_class("could you give me some advice for travling in France",model)[0]["intent"]

In [ ]:
predict_class("Can I restart my computer directly",model)[0]["intent"]

In [ ]:
predict_class("restart mysql service",model)[0]["intent"]

In [83]:

def get_max5(arr):

    ixarr = []
    for ix, el in enumerate(arr):
        ixarr.append((el, ix))
    ixarr.sort()
    ixs = []
    for i in ixarr[-5:]:
        ixs.append(i[1])
    return ixs[::-1]

In [84]:
oral = "Bot: Hi, Nice to hear from you!"
def feedback(n,limit):
    if(n==limit):
        print("Sorry about that, I am not sure I understand...")
        return
    else:
        global data
        question = questionset["question"][questionset.index[inds[n+1]]]
        answer =  data['answer'][questionset.index[inds[n+1]]]
        print("==="*90)
        print("Bot: "+data['answer'][questionset.index[inds[n+1]]])
        print("==="*90)
        usr = input("Was this answer helpful? Yes/No: ").lower().strip()
        if usr == 'yes':
            global oral 
            oral = "What can I help you now?"
            data = data.append({'question': original.strip().lower(), 'answer': answer, 'tag': pred_tag}, ignore_index=True)
            data.to_csv(csv_file_name,index=False)
            over = True
            return
        elif usr == "no":
            feedback(n+1,limit)
        else:
            print(".........????????")
            over = True
            oral = "Bot: Hi, Nice to hear from you!"
            return

In [85]:
stemmer = LancasterStemmer()
def cleanup(sentence):
    word_tok = nltk.word_tokenize(sentence)
    stemmed_words = [stemmer.stem(w) for w in word_tok]

    return ' '.join(stemmed_words)
le = LE()
tfv = TfidfVectorizer(min_df=1, stop_words='english')
questions = data['question'].values
X = []
for question in questions:
    X.append(cleanup(question))
tfv.fit(X)

TfidfVectorizer(stop_words='english')

In [ ]:
try:
    with open('username.pkl', 'rb') as file:  
        username = pickle.load(file)
        oral = "Bot: Hi "+username+", Nice to hear from you!"
except:
    oral = 'Hello,may I have your name?'
    print(oral)
    sentence_name = input("You: ")
    doc = nlp(sentence_name)
    if len(doc.ents)<=0:
        print("Maybe I can only name you sweetie :p")
        username = "sweetie"
    else:
        for ent in doc.ents:
            if(ent.label_ =="PERSON"):
                username = ent.text
            else:
                print("Maybe I can only name you sweetie :p")
                username = "sweetie"
    pickle.dump(username,open('username.pkl','wb'))
oral = "Bot: Hi "+username+", Nice to hear from you!"
print(oral)
while True:
    over = False
    print("PRESS Q to QUIT")
    original = input("You: ")
    if original.strip().lower()=="q":
        print("Bot: It was good to be of help, ",username)
        break
    else:
        pred_tag = predict_class(original.strip().lower(),model)[0]["intent"]
        questionset = data[data['tag'] == pred_tag]
        cos_sims = []
        t_original = tfv.transform([cleanup(original.strip().lower())])
        for question in questionset['question']:
            sims = cosine_similarity(tfv.transform([question]), t_original)
            cos_sims.append(sims)
        inds = get_max5(cos_sims)
        if over==False:
            feedback(0,3)
        else:
            continue

Hello,may I have your name?
You: My name is Ben
Bot: Hi Ben, Nice to hear from you!
PRESS Q to QUIT
You: Could you give me some advice for travling in the UK
Bot: I hope I figured it out. I have no more Black screen issue &amp; flickering since I did below  yesterday:


Found advise from a user and Killed process called "warmd", process user: unknown.
http://developer.apple.com/library/mac/#documentation/Darwin/Reference/ManPages/man8/warmd.8.html
When my Screen went black sporadically, My ipad was connected and nearby my MacBook , I relocated it further away (I read about interference issues)
I right clicked on battery icon and it showed me that Battery needs Service. I fully re-charged and re-booted my Pro and WoohLah, Battery status was back to normal....
I switched GPU >> Installed programm called GFXcardstatus (you can Google that)
In Applications->Utilities->Terminal
enter: sudo pmset -a lidwake 0   to reverse replace 0 with 1
enter your computer password...


This sends your lap